# Laden der Daten
In diesem Skript werden die Daten, die in der Arbeit genutzt werden geladen. Dabei werden die Tankstellen mit den Preisen kombiniert. Zudem wird gefiltert nach der Stadt Minden, die ausschließlich betrachtet werden soll und zu vollen Stunden zusammengefasst.

In [12]:
import pandas as pd
import glob
import json
import numpy as np

## Definition der Funktionen <a class="anchor" id="chapter1"></a>

 Alle .csv-Dateien in dem Unterordner werden zusammengefügt

 Output: DataFrame allen Tankstellen in Deutschland

In [13]:
csv_files = glob.glob('Stationen2024/**/*.csv')
print(csv_files)

['Stationen2024/03/2024-03-21-stations.csv', 'Stationen2024/03/2024-03-26-stations.csv', 'Stationen2024/03/2024-03-02-stations.csv', 'Stationen2024/03/2024-03-05-stations.csv', 'Stationen2024/03/2024-03-10-stations.csv', 'Stationen2024/03/2024-03-17-stations.csv', 'Stationen2024/03/2024-03-27-stations.csv', 'Stationen2024/03/2024-03-20-stations.csv', 'Stationen2024/03/2024-03-16-stations.csv', 'Stationen2024/03/2024-03-11-stations.csv', 'Stationen2024/03/2024-03-04-stations.csv', 'Stationen2024/03/2024-03-03-stations.csv', 'Stationen2024/03/2024-03-09-stations.csv', 'Stationen2024/03/2024-03-08-stations.csv', 'Stationen2024/03/2024-03-18-stations.csv', 'Stationen2024/03/2024-03-29-stations.csv', 'Stationen2024/03/2024-03-19-stations.csv', 'Stationen2024/03/2024-03-28-stations.csv', 'Stationen2024/03/2024-03-30-stations.csv', 'Stationen2024/03/2024-03-22-stations.csv', 'Stationen2024/03/2024-03-25-stations.csv', 'Stationen2024/03/2024-03-13-stations.csv', 'Stationen2024/03/2024-03-14-st

In [14]:
def laden_Stationen():
    csv_files = glob.glob('Stationen2024/**/*.csv')
    combined_df = pd.DataFrame()
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        combined_df = pd.concat([combined_df, df])
        combined_df = combined_df.drop_duplicates().reset_index(drop=True)
    return combined_df

Extrahieren aller Tankstellen einer Stadt aus allen Tankstellen in Deutschland

Input: Name der Stadt, DataFrame mit allen Tankstellen

Output: DataFrame mit Tankstellen einer Stadt

In [15]:
def extract_Stadt_Station(Stadt, df):
    dfStadt = df[df['city'] == Stadt]
    dfStadt = dfStadt.drop_duplicates(subset='uuid')
    dfStadt = dfStadt.rename(columns={'uuid': 'station_uuid'})
    return dfStadt

Generieren einer Liste aus den Tankstellen ID

Input: DataFrame mit den Tankstellen, Spaltenname mit der ID

Output: Liste mit den IDs der Tankstellen

In [16]:
def get_Liste_StationID(df, id):
    df.drop_duplicates(subset=id)
    list = df[id]
    return list

Alle .csv-Dateien in dem Unterordner werden zusammengefügt und nach ID-Liste sortiert

Output: DataFrame allen Tankstellen in Deutschland

In [17]:

def laden_Preise(ID):
    csv_files = glob.glob('Preise2024/**/*.csv')
    combined_df = pd.DataFrame()
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        df = df[df['station_uuid'].isin(ID)]
        combined_df = pd.concat([combined_df, df])
        combined_df = combined_df.drop_duplicates().reset_index(drop=True)
    return combined_df

Gruppierung nach Preisänderungen nach dem Mittelwert der Stunden

Input: Zusammengefügter DataFrame Preis und Tankstellen

Output: DataFrame mit gruppierten Stunden

In [18]:
def gruppieren_Stunden(df):
    df_meta = df[['station_uuid', 'brand', 'city', 'street', 'house_number', 'name', 'post_code', 'latitude',
                  'longitude', 'first_active', 'openingtimes_json']].drop_duplicates()
    df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True)
    df = df.dropna(subset=['date'])
    df['hour'] = df['date'].dt.floor('H')
    df_hourly = df.groupby(['station_uuid', 'hour'])[['diesel', 'e5', 'e10', 'dieselchange', 'e5change', 'e10change']].mean().reset_index()
    df_hourly = pd.merge(df_hourly, df_meta, on='station_uuid', how='left')
    return df_hourly

## Ausführung <a id="chapter2"></a>

In [19]:
dfStation = extract_Stadt_Station('Minden', laden_Stationen())
listStationID = get_Liste_StationID(dfStation, 'station_uuid')
dfPreise = laden_Preise(listStationID)
dfStationPreise = pd.merge(dfStation, dfPreise, on='station_uuid', how='inner')
dfStationPreise = gruppieren_Stunden(dfStationPreise)
#dfStationPreise.to_csv('df_PreiseStationen.csv')

/var/folders/_9/fyqnr96j0q7_tlytmj22wy8c0000gn/T/ipykernel_83299/1044408052.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['hour'] = df['date'].dt.floor('H')


# Weitere Verarbeitung

In [20]:
# Neue Spalte mit Datum
dfStationPreise['date'] = dfStationPreise['hour']
dfStationPreise["date"] = pd.to_datetime(dfStationPreise["hour"], errors="coerce")
# Wochentagen aus den Daten
dfStationPreise["weekday"] = dfStationPreise["date"].dt.day_name()
dfStationPreise["date"] = dfStationPreise["date"].dt.date
# Hour mit ausschließlich der Uhrzeit
dfStationPreise["hour"] = pd.to_datetime(dfStationPreise["hour"], errors="coerce")
dfStationPreise["hour"] = dfStationPreise["hour"].dt.time

# Change auf volle Stunden runden, sodass eine Änderung angezeigt werden kann
dfStationPreise['dieselchange'] = np.ceil(dfStationPreise['dieselchange'])
dfStationPreise['e5change'] = np.ceil(dfStationPreise['e5change'])
dfStationPreise['e10change'] = np.ceil(dfStationPreise['e10change'])

In [21]:
# Quelle: https://docs.python.org/3/library/json.html
# Extrahieren der Öffnungszeiten im JSON Format, neue Spalten für die Öffnung und das Schließen der Tankstelle
day_bit_values = {'Monday': 1,'Tuesday': 2,'Wednesday': 4,'Thursday': 8,'Friday': 16,'Saturday': 32,'Sunday': 64}
holiday_bit_value = 128

# Gesetzliche Feiertage in Minden
all_public_holidays = {
    '2023-12-25', '2023-12-26', '2024-01-01', '2024-03-29', '2024-04-01', '2024-05-01', '2024-05-09', '2024-05-20', '2024-05-30', '2024-10-03', '2024-11-01', '2024-12-25', '2024-12-26'}

def decode_opening_times(row):
    date_str = row['date']
    weekday_str = row['weekday']
    opening_times_json = row['openingtimes_json']
    start_time, end_time = None, None
    is_holiday = date_str in all_public_holidays
    current_day_bit = holiday_bit_value if is_holiday else day_bit_values.get(weekday_str)
    if current_day_bit is None:
        return pd.Series([start_time, end_time], index=['start_time', 'end_time'])
    try:
        opening_times_data = json.loads(opening_times_json)
        for rule in opening_times_data.get('openingTimes', []):
            if (rule.get('applicable_days', 0) & current_day_bit) == current_day_bit:
                periods = rule.get('periods')
                if periods:
                    start_time = periods[0].get('startp')
                    end_time = periods[0].get('endp')
                    break
    except (json.JSONDecodeError, TypeError, AttributeError):
        pass
    return pd.Series([start_time, end_time], index=['start_time', 'end_time'])

dfStationPreise[['start_time', 'end_time']] = dfStationPreise.apply(decode_opening_times, axis=1)

## Ergebniss <a id="chapter3"></a>

In [22]:
print(dfStationPreise.info())
print(dfStationPreise.head())
dfStationPreise.to_csv('df_PreiseStationen.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73589 entries, 0 to 73588
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   station_uuid       73589 non-null  object 
 1   hour               73589 non-null  object 
 2   diesel             73589 non-null  float64
 3   e5                 73589 non-null  float64
 4   e10                73589 non-null  float64
 5   dieselchange       73589 non-null  float64
 6   e5change           73589 non-null  float64
 7   e10change          73589 non-null  float64
 8   brand              68097 non-null  object 
 9   city               73589 non-null  object 
 10  street             73589 non-null  object 
 11  house_number       67120 non-null  object 
 12  name               73589 non-null  object 
 13  post_code          73589 non-null  object 
 14  latitude           73589 non-null  float64
 15  longitude          73589 non-null  float64
 16  first_active       735